In [2]:
import requests
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [1]:
url = 'http://10.12.105.245:8080/shot.jpg'
# save_folder = 'images'
# saved_image_path = get_image(url, save_folder)

In [ ]:
howmanypics = 10
personName = "Aditya"
save_path_train = "trainFaces"
save_path_test = "testFaces"

In [ ]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    if len(faces) != 1:
        return None
    x, y, w, h = faces[0]
    cropped_face = img[y: y+h, x:x+w]
    return cropped_face

def process_images(url_template, save_path_train, save_path_test, personName):
    for i in range(1, howmanypics+1):  # Process 100 images
        url = url_template.format(i)
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, -1)

        # Check if a face is detected in the first image
        if i == 1:
            face = face_cropped(img)
            if face is None:
                print("No face detected in the first image. Exiting.")
                return
        
        # Process and save the face as a 64x64 image
        face = face_cropped(img)
        face_resized = cv2.resize(face, (128, 128))
        face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
        if(i<int(howmanypics*0.7)):
            file_path = os.path.join(save_path_train, f'{personName}_{i}.jpg')
            cv2.imwrite(file_path, face_gray)
        else:
            file_path = os.path.join(save_path_test, f'{personName}_{i}.jpg')
            cv2.imwrite(file_path, face_gray)

    print("Faces cropped and saved successfully.")

# Example usage # URL template for 100 images
save_path = 'images'  # Specify where to save the cropped faces
process_images(url, save_path_train, save_path_test, personName)

In [ ]:
# Function to load and preprocess images from a folder
def load_images_from_folder(folder, image_size):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename))
            img = cv2.resize(img, image_size)
            images.append(img)
            labels.append(filename.split('_')[0])  # Assuming filename format: "PersonName_index.jpg"
    return np.array(images), np.array(labels)

# Set your dataset directories and image size
train_dir = 'trainFaces'
test_dir = 'testFaces'
image_size = (128, 128)

# Load training and testing images and labels
X_train, y_train = load_images_from_folder(train_dir, image_size)
X_test, y_test = load_images_from_folder(test_dir, image_size)

# Encode labels as integers
label_mapping = {label: idx for idx, label in enumerate(np.unique(y_train))}
y_train = np.array([label_mapping[label] for label in y_train])
y_test = np.array([label_mapping[label] for label in y_test])

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_mapping), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
test_image = cv2.imread('2.jpg')
test_image = cv2.resize(test_image, image_size)
test_image = np.expand_dims(test_image, axis=0)
prediction = model.predict(test_image)
predicted_label = list(label_mapping.keys())[np.argmax(prediction)]
print(f"Predicted Label: {predicted_label}")

In [ ]:
def capture_images(url): #, interval_seconds=5, num_images=10
    image_list = []

    # for _ in range(num_images):
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    # img = imutils.resize(img, width=1000, height=1800)

    image_list.append(img)
        # time.sleep(interval_seconds)

    return image_list
# url_to_capture = "http://10.12.60.98:8080//shot.jpg"
# captured_images = capture_images(url=url_to_capture, interval_seconds=2, num_images=10)

def face_cropped_from_list(images_list):
    # Load the face classifier
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    all_faces_list = []

    for img in images_list:
        faces_list = []
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the image
        faces = face_classifier.detectMultiScale(gray, 1.2, 5)

        # Iterate through detected faces
        for (x, y, w, h) in faces:
            # Crop the face from the original image
            cropped_face = img[y: y+h, x:x+w]

            # Resize the face to a common size (e.g., 200x200)
            cropped_face = cv2.resize(cropped_face, (128, 128))

            # Convert the face to grayscale
            cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2GRAY)

            # Append the face to the faces_list
            faces_list.append(cropped_face)

        # Append the faces_list to the all_faces_list
        all_faces_list.append(faces_list)

    return all_faces_list, len(all_faces_list)

x, y = face_cropped_from_list(capture_images(url))
y